In [2]:
#!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
#!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip

import zipfile
with zipfile.ZipFile('/content/BelgiumTSC_Training.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
import torch.utils.data as td
import matplotlib.pyplot as plt
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F
import gc
import torchvision.datasets as datasets


test_path = '/content/Testing'
train_path = '/content/Training'

preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.RandomAutocontrast(),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

data = torchvision.datasets.ImageFolder(root=train_path, transform=preprocess)

train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=0)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True,  num_workers=0)

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 205):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

num_epochs = 30
batch_size = 64
learning_rate = 0.001
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = ResNet(ResidualBlock, [3, 4, 6, 3],62).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_data_loader)
accuracy_list = []
steps_list = []
train_loss = []
test_accuracy = []
test_loss = []

In [6]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_data_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        accuracy = (correct / total) * 100
        steps = i + 1
        accuracy = (correct / total) * 100
        accuracy_list.append(accuracy)
        steps_list.append(steps)
        train_loss.append(loss.item())


    del images, labels, outputs
    torch.cuda.empty_cache()
    gc.collect()
    print ('Training set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                .format(epoch+1, num_epochs, loss.item(), accuracy))

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss_test = criterion(outputs, labels)
            del images, labels, outputs
        accuracy = (correct / total) * 100    
        test_accuracy.append(accuracy)
        test_loss.append(loss_test.item())
        print ('Test set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                    .format(epoch+1, num_epochs, loss_test.item(), accuracy))


Training set: Epoch [1/30], Loss: 3.5520,  Accuracy: 16.67%, 
Test set: Epoch [1/30], Loss: 3.6346,  Accuracy: 14.54%, 
Training set: Epoch [2/30], Loss: 3.5752,  Accuracy: 16.67%, 
Test set: Epoch [2/30], Loss: 3.2235,  Accuracy: 22.73%, 
Training set: Epoch [3/30], Loss: 2.3841,  Accuracy: 66.67%, 
Test set: Epoch [3/30], Loss: 2.5257,  Accuracy: 30.71%, 
Training set: Epoch [4/30], Loss: 2.1417,  Accuracy: 66.67%, 
Test set: Epoch [4/30], Loss: 3.0442,  Accuracy: 34.97%, 
Training set: Epoch [5/30], Loss: 3.4692,  Accuracy: 8.33%, 
Test set: Epoch [5/30], Loss: 2.9213,  Accuracy: 38.36%, 
Training set: Epoch [6/30], Loss: 2.5362,  Accuracy: 50.00%, 
Test set: Epoch [6/30], Loss: 2.4214,  Accuracy: 41.53%, 
Training set: Epoch [7/30], Loss: 1.9589,  Accuracy: 41.67%, 
Test set: Epoch [7/30], Loss: 2.3443,  Accuracy: 44.48%, 
Training set: Epoch [8/30], Loss: 2.5562,  Accuracy: 41.67%, 
Test set: Epoch [8/30], Loss: 2.4804,  Accuracy: 45.68%, 
Training set: Epoch [9/30], Loss: 2.7534,

In [7]:
PATH = '/content/RESNET_BTSD.pt'
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
            'accuracy': accuracy_list,
            'test_loss': test_loss,
            'test_accuracy':test_accuracy
            }, PATH)

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH = '/content/gdrive/My Drive/RESNET_BTSD.pt'
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
            'accuracy': accuracy_list,
            'test_loss': test_loss,
            'test_accuracy':test_accuracy
            }, PATH)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
